# Compare success of different lineages
Natalia Vélez, July 2020

Now that we've built a graph representation of family trees, we'll use it to compare lineages and operationalize a success measure.

In [1]:
%matplotlib inline

import os, re, glob, datetime, json
from os.path import join as opj

import pandas as pd
import numpy as np
import scipy.stats
from tqdm import notebook

import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from networkx.drawing.nx_agraph import graphviz_layout

sns.set_context('talk')
sns.set_style('white')

## Set up data

Find start of boundless era from version history:

In [2]:
ver_history = pd.read_csv('../1_download/outputs/version_history.tsv', sep='\t')
boundless_t = ver_history.loc[ver_history['release'] == 280, 'timestamp'].values[0]
print('Start of boundless: %s' % datetime.datetime.fromtimestamp(boundless_t))
ver_history.head()

Start of boundless: 2019-11-08 17:27:14


,release,timestamp
0,1,1.483052e+09
1,5,1.483472e+09
2,8,1.484065e+09
3,14,1.484961e+09
4,16,1.492207e+09


Load lifelog data:

In [3]:
all_eras = pd.read_csv('outputs/all_lifelogs_compact.tsv', sep='\t', index_col=0)
all_eras.head()

/n/home15/nvelezalicea/.conda/envs/py3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,player,avatar,tBirth,parent,birth,tDeath,death,age,cause_of_death,birthX,birthY,deathX,deathY,first,last
0,5ab4f808b01db8bab564fa726ca7dd0439205d4a,4108849,1617678319,4108781,[-13743 -77],1.617679e+09,[-13797 -104],3.59,hunger,-13743,-77,-13797.0,-104.0,TED,GFELL
1,5ab4f808b01db8bab564fa726ca7dd0439205d4a,4108848,1617678300,4108786,[-15498 362],1.617678e+09,[-15498 362],0.22,disconnect,-15498,362,-15498.0,362.0,HERA,POLLY
2,dadea1a71832326c598df59059cf42102e979707,4108842,1617678238,4108778,[-15498 362],1.617680e+09,[-15502 363],22.16,murdered,-15498,362,-15502.0,363.0,SPARTAN,POLLY
3,328dc412f542693dc20d084e99a7367e4fe4aae4,4108839,1617678208,4108784,[-13754 -85],1.617682e+09,[-13765 -64],60.00,oldAge,-13754,-85,-13765.0,-64.0,SERANA,GFELL
4,e65b7bfa037a3287869cb682b648c68e52ad05d4,4108837,1617678173,4108782,[-15796 -205],1.617679e+09,[-15803 -204],14.85,murdered,-15796,-205,-15803.0,-204.0,TJ,NaN


Load families & find families that started in boundless era:

In [4]:
fam_df = pd.read_csv('outputs/family_playerID.tsv', sep='\t', index_col=0)
fam_df.head()

/n/home15/nvelezalicea/.conda/envs/py3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,avatar,family
0,4108692,time-1617674434_eve-4108692_name-NOTIS
1,4108622,time-1617672864_eve-4108622_name-JONES
2,4108226,time-1617661499_eve-4108226_name-SAD
3,4108225,time-1617661497_eve-4108225_name-LOSTAUNAU
4,4108220,time-1617661402_eve-4108220_name-AA


Find families that started in boundless era:

In [5]:
in_boundless = fam_df.groupby('family')['avatar'].agg('count').reset_index()
in_boundless['start_t'] = in_boundless['family'].str.extract(r'(?<=time-)([0-9]+)')
in_boundless['start_t'] = in_boundless['start_t'].astype(np.int)
in_boundless['in_era'] = in_boundless['start_t'] > boundless_t
in_boundless = in_boundless[['family', 'in_era']]

# Summary: How many families left?
print('Total families: %i' % in_boundless.shape[0])
in_boundless.groupby('in_era')['family'].agg('count')

Total families: 64863


in_era
False    49880
True     14983
Name: family, dtype: int64

Merge:

In [6]:
print('Original size: %s' % str(all_eras.shape))
boundless_df = pd.merge(all_eras, fam_df, on='avatar')
boundless_df = pd.merge(boundless_df, in_boundless, on='family')
boundless_df = boundless_df[boundless_df['in_era']]
boundless_df = boundless_df.reset_index(drop=True)
print('Boundless era only: %s' % str(boundless_df.shape))

Original size: (2226610, 15)
Boundless era only: (789389, 17)


Remove singleton families:

In [7]:
singletons = boundless_df.groupby('family')['avatar'].agg('count').reset_index()
singletons = singletons.rename(columns={'avatar': 'count'})
singletons = singletons[singletons['count'] == 1]
single_fams = singletons.family.values

print('Removing %i singleton families' % len(single_fams))
boundless_df = boundless_df[~boundless_df['family'].isin(single_fams)]
#era_df = era_df.reset_index(drop=True)

Removing 8426 singleton families


Remove incomplete families (i.e., lineages that were still alive at the time of the data download)

In [8]:
idx = boundless_df.groupby(['family'])['tBirth'].transform(max) == boundless_df['tBirth']
last_descendants = boundless_df[idx].copy()
incomplete_fams = last_descendants[~np.isfinite(last_descendants.tDeath)].family.values

print('Removing %i incomplete families' % len(incomplete_fams))
boundless_df = boundless_df[~boundless_df['family'].isin(incomplete_fams)]

Removing 11 incomplete families


Exclude infant births:

In [9]:
orig_n = len(boundless_df)
infant_deaths = np.loadtxt('outputs/infant_deaths.txt', dtype=np.int)
boundless_df = boundless_df[~boundless_df['avatar'].isin(infant_deaths)].reset_index(drop=True)
new_n = len(boundless_df)

print('Removed %i infant deaths' % (orig_n - new_n))

OSError: outputs/infant_deaths.txt not found.

Just look at lineages from the boundless world era:

In [ ]:
boundless_families = np.unique(boundless_df['family'])

print('Analyzing %i families' % len(boundless_families))
print('%i avatars' % len(np.unique(boundless_df['avatar'])))
print('%i unique players' % len(np.unique(boundless_df['player'])))

Lives per player in current sample:

In [ ]:
lineage_df = boundless_df.copy()
player_summ = lineage_df.groupby('player')['avatar'].agg('count').reset_index()
player_summ = player_summ.rename(columns={'avatar': 'n'})
player_summ.head()

# Plot
print('Median # lives: %0.2f' % np.median(player_summ['n']))
print('Min # lives: %i' % np.min(player_summ['n']))
print('Max # lives: %i' % np.max(player_summ['n']))

fig,ax = plt.subplots()
sns.distplot(np.log10(player_summ['n']),ax=ax)
ax.set(xlabel='# lives (excluding disconnects)', xticks=range(4), xticklabels=[10**x for x in range(4)])
sns.despine()

Start and end times:

In [ ]:
t_fmt = '%Y-%m-%dT%H:%M:%S %Z'

start_t = np.min(lineage_df['tBirth'])
start_date = datetime.datetime.fromtimestamp(start_t).strftime(t_fmt)

end_t = np.max(lineage_df['tDeath'])
end_date = datetime.datetime.fromtimestamp(end_t).strftime(t_fmt)

print('Analyzing %i lineages' % len(boundless_families))
print('First lineage starts at: %s' % start_date)
print('Lineages end at: %s' % end_date)

What is the longest-lived lineage?

In [ ]:
family_t = lineage_df.groupby('family').agg({'tBirth': 'min', 'tDeath': 'max'}).reset_index()
family_t['duration_hours'] = (family_t['tDeath'] - family_t['tBirth'])/60/60
family_t['duration_days'] = family_t['duration_hours']/24
family_t = family_t.sort_values(by='duration_hours', ascending=False).reset_index(drop=True)
family_t.head()

In [ ]:
ax = sns.distplot(np.log10(family_t.duration_hours))
log_xticks = ax.get_xticks()
log_xticks = log_xticks[1:-1]
ax.set(xticks = log_xticks, xticklabels = [10**x for x in log_xticks],
       xlabel = 'Family duration (hours)')

In [ ]:
family_t['duration_hours'].agg(['min', 'max', 'median'])

## Compute summary statistics

### Family size and life expectancy

In [ ]:
life_expectancy = lineage_df.groupby('family')['age'].agg(['mean', 'count']).reset_index()
ax = sns.distplot(life_expectancy['mean'])
ax.set(xlabel = 'Life expectancy by family')
sns.despine()

In [ ]:
ax = sns.distplot(np.log10(life_expectancy['count']))
ax.set(xlabel = 'Total family size', xticks=range(5), xticklabels=[10**i for i in range(5)])
sns.despine()

In [ ]:
scipy.stats.mode(life_expectancy['count'])

### Living population size over time

In [ ]:
living_list = []
for fam in notebook.tqdm(boundless_families):
    fam_df = lineage_df[lineage_df['family'] == fam].copy().reset_index(drop=True)
    t = fam_df['tBirth'].values
    for ti in t:
        is_alive = (fam_df['tBirth'] <= ti) & (fam_df['tDeath'] > ti)
        pop = np.sum(is_alive)
        living_list.append((fam, ti, pop))

In [ ]:
living_df = pd.DataFrame(living_list, columns = ['family', 't', 'population'])
living_df = living_df.sort_values(by=['family', 't'], ascending=True).reset_index(drop=True)
living_df['t0'] = living_df.groupby('family')['t'].transform('first')
living_df['t_elapsed'] = (living_df['t'] - living_df['t0'])/60/60
living_df.head()

Plot a subset

In [ ]:
np.random.seed(526)
random_families = np.random.choice(boundless_families, size=10, replace=False)
random_subset = living_df[living_df['family'].isin(random_families)]
g = sns.relplot(data=random_subset, x='t_elapsed', y='population', hue='family', kind='line',
                 height=6, aspect=2, alpha = 0.5)
g.set(xlabel = 'Time elapsed (hours)', ylabel = 'Population size')
g._legend.remove()

Maximum population size

In [ ]:
max_pop = living_df.groupby('family')['population'].agg('max').reset_index()
ax = sns.distplot(max_pop['population'])
ax.set(xlabel = 'Maximum living population size')
sns.despine()

In [ ]:
max_pop['population'].agg(['min', 'max', 'median'])

### # of generations (Chain length)

Helper: Read JSON files

In [ ]:
def open_graph(f):
    with open(f) as handle:
        graph_data = json.load(handle)
    return nx.json_graph.node_link_graph(graph_data)

Find longest chain in family graphs

In [ ]:
chain_list = []
family_generations = []

for f in notebook.tqdm(boundless_families):
    fam_file = 'outputs/families/families_%s.json' % f
    fam_graph = open_graph(fam_file)
    fam_chain = nx.algorithms.dag_longest_path(fam_graph)
    
    eve = re.search(r'(?<=eve-)([0-9]+)', f).group(0)
    
    chain_list.append((f, len(fam_chain)))

In [ ]:
chain_df = pd.DataFrame(chain_list, columns=['family', 'longest_path'])
chain_df.head()

In [ ]:
chain_df['longest_path'].agg(['min', 'max', 'median'])

In [ ]:
ax = sns.distplot(np.log10(chain_df['longest_path']))

ax.set(xlabel = '# of generations')


In [ ]:
chain_df.sort_values(by='longest_path', ascending=False).head()

## Modeling success

Criterion for "success": Reaching age 14 (viability fitness)

In [ ]:
mortality_df = lineage_df.copy()
mortality_df['adult'] = (lineage_df['age'] >= 14)*1
mortality_df = mortality_df[['family', 'avatar', 'adult']]

mortality_summ = mortality_df.groupby('family')['adult'].agg(['sum', 'count']).reset_index()
mortality_summ['sum'] = mortality_summ['sum'].astype(np.int)
mortality_summ.head()

Compute beta distribution for each family

Prior: Uniform
$$
\theta \sim \mathrm{Beta}(\alpha_0, \beta_0) \\
\alpha_0 = \beta_0 = 1
$$

Posterior-sufficient statistics:
$$
\theta | D \sim \mathrm{Beta}(\alpha_0 + k, \beta_0 + N - k) \\ 
a = \alpha_0 + k \\
b = \beta_0 + N - k
$$

In [ ]:
def beta_mean(row): return scipy.stats.beta.mean(row['a'], row['b'])
def beta_var(row): return scipy.stats.beta.var(row['a'], row['b'])

mortality_summ['a'] = 3 + mortality_summ['sum']
mortality_summ['b'] = 3 + mortality_summ['count'] - mortality_summ['sum']
mortality_summ['beta_mean'] = mortality_summ.apply(beta_mean, axis=1)
mortality_summ['beta_var'] = mortality_summ.apply(beta_var, axis=1)
mortality_summ['snr'] = mortality_summ['beta_mean']/mortality_summ['beta_var']
mortality_summ['weighted_size'] = mortality_summ['beta_mean']*mortality_summ['count']
mortality_summ = pd.merge(mortality_summ, chain_df, on = 'family')
mortality_summ.to_csv('outputs/family_fitness.tsv', sep='\t', index=False)
mortality_summ.head()

### Compare candidate success measures

Beta mean:

In [ ]:
ax = sns.distplot(mortality_summ['beta_mean'])
ax.set(xlabel = 'Mean viability ($\mu$)')
sns.despine()

Distribution of SNR:

In [ ]:
ax = sns.distplot(np.log10(mortality_summ['snr']))
ax.set(xlabel='Signal-to-noise ratio ($\mu/\sigma$)',
       xticks = np.arange(1,5), xticklabels=[10**x for x in np.arange(1,5)])
sns.despine()

Distribution of weighted size:

In [ ]:
ax = sns.distplot(np.log10(mortality_summ['weighted_size']))
ax.set(xlabel='Weighted family size ($\mu N$)',
       xticks=np.arange(0, 5), xticklabels=[10**x for x in np.arange(0, 5)])

Distribution of # adults:

In [ ]:
ax = sns.distplot(np.log10(mortality_summ['sum']))
ax.set(xlabel='# of adults', xticks = np.arange(0,5), xticklabels=[10**x for x in np.arange(0,5)])

### Plot representative families

Split data into quartiles:

In [ ]:
success = 'sum' # Success metric
mortality_summ['quantile'] = pd.qcut(mortality_summ[success], 4, labels=False)

What are the quantiles?

In [ ]:
success_q = scipy.stats.mstats.mquantiles(mortality_summ[success], prob=[0.25, 0.5, 0.75, 1])
mortality_summ['log_n'] = np.log10(mortality_summ['sum'])

# Plot!
q_log = np.log10(success_q)
plt.figure(figsize=(12,4))
for q in q_log:
    plt.axvline(q, color='#aaaaaa', linestyle='--')

ax = sns.distplot(mortality_summ['log_n'])
ax.set_xticks(range(5))
labels = [10**t for t in ax.get_xticks()]
ax.set(xlabel='# of adults', xticklabels=labels)
sns.despine()

Plot representative family trees from each quartile:

In [ ]:
np.random.seed(526)
representative_families = np.array([np.random.choice(group['family'], 10) 
                                    for name,group in mortality_summ.groupby('quantile')])
rep_list = np.array(representative_families)
rep_list = rep_list.flatten()

rep_info = mortality_summ.copy()
rep_info = rep_info[rep_info['family'].isin(rep_list)]
rep_info = rep_info.reset_index(drop=True)
rep_info = rep_info.sort_values('quantile')
rep_info.to_csv('plots/fitness_quantiles/selected_families.tsv', sep='\t', index=None)
rep_info.head()

In [ ]:
for quant in notebook.tqdm(range(4)):
    for f in notebook.tqdm(representative_families[quant]):
        fam_file = 'outputs/families/families_%s.json' % f
        out_file = 'plots/fitness_quantiles/families_Q%i_%s.png' % (quant+1, f)

        # Load graph and remove infant deaths
        fam_graph = open_graph(fam_file)

        # Figure size (based on graphviz layout)
        nx.nx_agraph.write_dot(fam_graph,'fam.dot')
        pos=graphviz_layout(fam_graph, prog='dot')
        pos_coords = pd.DataFrame(list(pos.values()), columns=['x','y']).agg(['max', 'min'])
        w = (pos_coords.loc['max', 'x'] - pos_coords.loc['min', 'x'])/150
        h = (pos_coords.loc['max', 'y'] - pos_coords.loc['min', 'y'])/150

        # Adjust for 2-member families
        w = max(w, 2)
        h = max(h, 2)

        # Node color (based on whether individuals reached maturity)
        fam_nodes = list(fam_graph.nodes)
        fam_nodes = [int(n) for n in fam_nodes]

        fam_attr = mortality_df[['avatar', 'adult']].copy()
        fam_attr = fam_attr[fam_attr['avatar'].isin(fam_nodes)]
        fam_attr = fam_attr.set_index('avatar')
        fam_attr = fam_attr.to_dict()
        
        fam_color = []
        for n in fam_nodes:
            if n in infant_deaths:
                fam_color.append('#cccccc')
            else:
                if fam_attr['adult'][n] == 1:
                    fam_color.append('#4ab1ff')
                else:
                    fam_color.append('#4ab1ff')

#         fam_color = [fam_attr['adult'][n] == 1 for n in fam_nodes]
#         fam_color = ['#4ab1ff' if c else '#cccccc' for c in fam_color]

        plt.figure(3,figsize=(w,h)) 
        nx.draw(fam_graph, pos, with_labels=False, arrows=True, node_color=fam_color)
        plt.savefig(out_file, transparent=True)
        plt.close()

## Plots for talk

<div style='background-color:red;color:white;'>TODO: The plots below should be moved elsewhere to preserve order!</div>
Relationship between family and repertoire size:

In [ ]:
rep_df = pd.read_csv('../3_technology/outputs/family_repertoire.tsv', sep='\t')
rep_df.head()

In [ ]:
family_n = mortality_summ[['family','sum', 'log_n']]
family_rep = rep_df[['family', 'breadth','log_breadth']]
n_rep = pd.merge(family_n, family_rep, on='family')
n_rep.head()

Max items possible?

In [ ]:
item_df = pd.read_csv('../4_techtree/num_unique_ingredients.csv')
n_items = len(item_df)
print('# items: %i' % n_items)
item_df.head()

Distribution of repertoire sizes

In [ ]:
rep_q = scipy.stats.mstats.mquantiles(rep_df['log_breadth'], prob=[0.25, 0.5, 0.75, 1])
print(rep_q)

plt.figure(figsize=(12,4))
# for q in rep_q:
#     plt.axvline(q, color='#aaaaaa', linestyle='--')
#plt.axvspan(0, q_log[0], alpha=0.5, color='red') 

ax = sns.distplot(rep_df['log_breadth'], bins=20)
ax.set_xlim(left=0)
ax.set_xticks(range(5))
plt.axvline(np.log10(n_items), linestyle='--', color = '#f5a442')
labels = [10**t for t in ax.get_xticks()]
ax.set(xlabel='Repertoire size', xticklabels=labels)
sns.despine()

In [ ]:
g = sns.jointplot(data=n_rep, x='log_breadth', y='log_n', kind='reg', lowess=True,
                  marginal_kws={'bins':20},
                  scatter_kws={'alpha':0.01, 'color': '#A5C8E1'})
ticks = np.arange(4)
tick_labels = [10**t for t in ticks]
g.ax_joint.set(xticks = ticks, yticks = ticks,
               xticklabels = tick_labels, yticklabels = tick_labels,
               xlabel='Viability',
               ylabel='Repertoire size')

In [ ]:
gini_df = pd.read_csv('../3_technology/outputs/family_gini.tsv', sep='\t')
gini_df.head()

In [ ]:
gini_rep = pd.merge(gini_df, rep_df, on='family')
gini_rep.head()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = sns.regplot(x='gini', y='log_breadth', data=gini_rep,
                 scatter_kws={'alpha': 0.05, 'color': '#A5C8E1'}, 
                 line_kws = {'color': '#2276B4'}, lowess=True)
yticks =  np.arange(0,5)
ax.set_yticks(yticks)
yticklabels = [10**y for y in yticks]
ax.set(xlabel = 'Innovation inequality (G)',
       xlim = (0,1),
       ylabel = 'Repertoire size',
       yticklabels=yticklabels)
sns.despine()

In [ ]:
plt.figure(figsize=(7,5))
equal_dist = np.array([0,1,2,3,4])
ax = sns.distplot(equal_dist, kde=False, hist_kws={'density': False}, bins=5)
ax.set(xlim = (0,11), ylim = (0,5),
       xticks = np.arange(0,11),
       yticks = np.arange(6),
       xlabel='# discoveries',
       ylabel='# players')
sns.despine()